In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
# openai work
from openai import OpenAI

client = OpenAI()
def stream_openai(client, role, message, model="gpt-4o-mini"):
    stream = client.chat.completions.create(
        model=model,
        messages=[{"role": role, "content": message}],
        stream=True,
    )
    message = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            message += chunk.choices[0].delta.content
    return message

# m = stream_openai(client, "user", "hello, how are you?")
# m

In [3]:
# Preprocessor test
from pathlib import Path

from ragna.source_storages import RagnaDemoSourceStorage
from ragna.core import (
    LocalDocument, Document
)


storage = RagnaDemoSourceStorage()

# md_files = 
# ragna_documents = [
#             (
#                 document
#                 if isinstance(document, Document)
#                 else LocalDocument.from_path(document)
#             )
#             for document in md_files
#         ]
# storage.store("ragna_docs", ragna_documents)

temporal_doc_paths = [p for p in Path.cwd().joinpath("temporal_docs").iterdir() if p.suffix == ".md"]
temporal_docs = [
    LocalDocument.from_path(path) for path in temporal_doc_paths
]
storage.store("temporal_docs", temporal_docs)

## Run ragna with no preprocessing on temporal_docs

In [4]:
from typing import Optional

from ragna.assistants import RagnaDemoAssistant

from ragna import Rag
from ragna.assistants import Gpt4


chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="temporal_docs",
)
_ = await chat.prepare()
print(await chat.answer("What happened last year?"))

Last year, in 2023, a rocket was built.


## Run with Preprocessor

In [5]:
from typing import Optional
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)

base_prompt = (
"""You are a llm agent that is responsible for rewriting an input prompt for a RAG application. 
A question might contain hidden context that may not be recognized by the embedding model. 
An embedding of a question might not generate a close match to an embedding of a statement
that contains the answer to the question. Although this is technically something that should
be solved on the embedding model side, it is usually solved by rephrasing the question
before using it to retrieve sources. For example a question like "What happened last month?" 
likely won't get any close matches. Rephrasing the prompt to "What happened in December 2021?"
given that the question is asked January 2022. Things that may be important to consider when 
reworking the prompt would be the current context, that may that is not explicitly asked in the 
question but can be inferred, for instance the current date. 

current date: 2025-01-24


Please reword the following prompt 
and only return the reworded prompt. I do not need to know your reasoning:


""")

class TestPreprocessor(QueryPreprocessor):

    def __init__(self):
        # self.storage=storage
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):
        instruction = (base_prompt + prompt)
        print(instruction)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        processed_query = self.ask_assistant(query)
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [9]:
preprocessor = TestPreprocessor()
processed_query = preprocessor.process("What happened last year?", None)

You are a llm agent that is responsible for rewriting an input prompt for a RAG application. 
A question might contain hidden context that may not be recognized by the embedding model. 
An embedding of a question might not generate a close match to an embedding of a statement
that contains the answer to the question. Although this is technically something that should
be solved on the embedding model side, it is usually solved by rephrasing the question
before using it to retrieve sources. For example a question like "What happened last month?" 
likely won't get any close matches. Rephrasing the prompt to "What happened in December 2021?"
given that the question is asked January 2022. Things that may be important to consider when 
reworking the prompt would be the current context, that may that is not explicitly asked in the 
question but can be inferred, for instance the current date. 

current date: 2025-01-24


Please reword the following prompt 
and only return the reworded prompt. 

In [10]:
print(processed_query.processed_query)

What happened in November 2024?


In [8]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="temporal_docs",
)
_ = await chat.prepare()
print(await chat.answer(processed_query.processed_query))

In 2024, they went to the moon.


# query preprocessor test

In [33]:
# Ragna docs
from ragna.source_storages import LanceDB

docs_path = Path.cwd().joinpath("../docs")

md_files = list(docs_path.glob("**/*.md"))
print(md_files[:2])

storage = LanceDB()

documents = [
            (
                document
                if isinstance(document, Document)
                else LocalDocument.from_path(document)
            )
            for document in md_files
        ]
storage.store("ragna_docs", documents)

[PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/preprocessing_demo/../docs/install.md'), PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/preprocessing_demo/../docs/index.md')]


/home/andrew/micromamba/envs/ragna-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
example_question = "how do I build a rag application?"

In [76]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="ragna_docs",
)
_ = await chat.prepare()
print(await chat.answer(example_question))

I'm sorry, but the information provided does not include details on how to build a RAG application.


In [77]:
from typing import Optional
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)

base_prompt = (
"""You are a helpful software engineer. Hypothetically given a specific software package in python,
 how would you answer the following question from a more junior developer?

 question:
""")

class QueryExpansionPreprocessor(QueryPreprocessor):

    def __init__(self, storage):
        self.storage=storage
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):
        instruction = (base_prompt + prompt)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        hypothetical_answer = self.ask_assistant(query)
        sources = self.storage.retrieve("ragna_docs", metadata_filter, hypothetical_answer)
        
        processed_query = "\n".join(s.content for s in sources) + "\n\n" + query
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [78]:
preprocessor = QueryExpansionPreprocessor(storage)

preprocessed_query = preprocessor.process(example_question, None)
preprocessed_query


ProcessedQuery(original_query='how do I build a rag application?', processed_query='# Frequently asked questions\n\n## Why should I use Ragna and not X?\n\n!!! tip "TL;DR"\n\n    Ragna is the only tool out there that specializes in orchestrating RAG use cases\n    with arbitrary components, as well as offering a Python API, a REST API, and a web\n    UI for that.\n\n!!! note\n\n    Although we try to be objective as possible, this section is inheritly biased. If\n    you are the author of a package we don\'t mention below but think we should or your\n    package is mentiond but you feel we have mischaracterized it, please\n    [get in touch](https://github.com/Quansight/ragna/discussions).\n\nAfter the emergence of ChatGPT in November of 2022, the field of LLMs exploded. Today,\nthere are many providers for LLM REST APIs out there. With these, we also have a\nplethora of Python packages to build applications around the provided APIs. We cannot\nsummarize the whole field here, so we sti

In [79]:
preprocessed_query

ProcessedQuery(original_query='how do I build a rag application?', processed_query='# Frequently asked questions\n\n## Why should I use Ragna and not X?\n\n!!! tip "TL;DR"\n\n    Ragna is the only tool out there that specializes in orchestrating RAG use cases\n    with arbitrary components, as well as offering a Python API, a REST API, and a web\n    UI for that.\n\n!!! note\n\n    Although we try to be objective as possible, this section is inheritly biased. If\n    you are the author of a package we don\'t mention below but think we should or your\n    package is mentiond but you feel we have mischaracterized it, please\n    [get in touch](https://github.com/Quansight/ragna/discussions).\n\nAfter the emergence of ChatGPT in November of 2022, the field of LLMs exploded. Today,\nthere are many providers for LLM REST APIs out there. With these, we also have a\nplethora of Python packages to build applications around the provided APIs. We cannot\nsummarize the whole field here, so we sti

In [80]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="ragna_docs",
)
_ = await chat.prepare()
print(await chat.answer(preprocessed_query.processed_query))

To build a RAG (Retrieval-Augmented Generation) application using Ragna, you would need to follow these steps:

1. Install Ragna: You can install Ragna using pip, a package installer for Python. You can do this by running the command `pip install ragna` in your terminal.

2. Set up your configuration: Ragna uses a configuration file to set up the environment. This includes setting up the local root directory for storing files, the authentication class for user authentication, the key-value store class for temporary storage, and the document class for uploading and reading documents. You can also set up the source storages and assistants available for the user to use.

3. Write your RAG components: You would need to write your own RAG components, which include the document, source storage, and assistant classes. These classes should inherit from the respective base classes in Ragna and implement the required methods.

4. Run your application: Once you have set up your configuration and 